In [ ]:
#@title ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title データを保存するディレクトリのパスを指定
#@markdown ランタイムに接続しドライブを後、コピーしてくる<br>
#@markdown 後ろの / は追加するので、追記は不要<br>

cwd = "/content/drive/MyDrive/RNAseq/Outputs/\u5B9F\u9A13\u540D" #@param {type:"string"}
cwd = cwd + "/"

# 必要なモジュールをインポートする
import pandas as pd
import numpy as np
import urllib.parse
from google.colab import files

# カレントディレクトリの変更
import os
os.makedirs(cwd , exist_ok = True)
os.chdir(cwd)
currentdir = os.getcwd()

In [ ]:
#@title データの読み込み、区切り文字の変更に伴う列名の再定義を行う
data_path = "Kallisto\u306E\u51FA\u529B\u3067\u3042\u308Babundance.tsv\u306E\u30D1\u30B9" #@param {type:"string"}

cols = ["ENST" , "ENSG" , "OTTHUMG" , "OTTHUMT" , "transcript" , "Gene" , "Len" , "data" , "target_id" , "Length" , "eff_Length" , "est_counts" , "tpm"]
df1  = pd.read_csv(data_path ,
                   sep = "[\t |]" ,
                   engine="python" ,
                   names = cols ,
                   skiprows = 1 ,)

df1 = df1.drop(["Len" , "target_id"] , axis = 1)
df1.to_csv("Kallisto_whole_data_tpm.csv" , sep="," , index = False)
df1

In [ ]:
#@title est_counts>0のデータで絞り込み
df2 = df1[df1.est_counts > 0]
df2.reset_index(drop = True , inplace = True)
df2.to_csv("Kallisto_no0tpm.csv" , sep="," , index = False)
df2

必要行スライスの取得、Lambda sumによるTPM値の合算

In [ ]:
#@title 同名GeneのTPM合計値を算出
#@markdown サンプル名を入力
sample_name = "\u30B5\u30F3\u30D7\u30EB\u540D\u3092\u5165\u529B" #@param {type:"string"}
df3 = df2.loc[: , ["Gene" , "tpm"]]
df3 = df3.groupby("Gene").apply(lambda x: x.sum()).drop("Gene",axis=1)

df3.rename(columns = {"tpm" : sample_name} , inplace = True)

df4 = df2.loc[: , ["ENSG" , "Gene"]]
for i in df4.ENSG:
  id = i.split(".")[0]
  df4.replace(i , id , inplace = True)
  
# print(df4)

df5 = pd.merge(df4 , df3 , on = "Gene" , how = "inner")
df5.set_axis(["Geneid" , "product" , sample_name] , axis = 1 , inplace = True)
df5.set_index("Geneid" , inplace = True)
df5.to_csv("TPMsum.csv" , sep = "," )
df5